In [61]:
import sys, os, django
import pandas as pd

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *

from pyparsing import *

q = Query.objects.get(pk=7638)
t = Tag.objects.get(pk=4366)

#q = Query.objects.get(pk=7507)
#t = Tag.objects.get(pk=4329)


In [95]:
doids = set(DocOwnership.objects.filter(tag=t).values_list('doc__id',flat=True))
uids = set(DocOwnership.objects.filter(tag=t).values_list('user__id',flat=True))

users = User.objects.filter(pk__in=uids)

docs = Doc.objects.filter(pk__in=doids)#, docproject__project=q.project, docproject__relevant__in=[1,3])
db2 = Category.objects.get(id=362)
print(db2)

docs.count()

Does this document fulfill all Level 1 AND 2 criteria for inclusion in Database 2<hidden><wholetext>


100

In [132]:
doc_dict = []

for d in docs.values("title","content","id"):
    db1_cols = []
    db2_cols = []
    note_cols = []
    d = dict(d)
    db2s = []
    d["link"] = f"https://apsis.mcc-berlin.net/scoping/document/193/{d['id']}/"
    d["db1_overall"] = DocProject.objects.get(project=q.project, doc_id=d["id"]).get_relevant_display()
    for u in users:   
        uname = u.username.split("@")[0]
        db2_cols.append(f"{uname} db2")
        db1_cols.append(f"{uname} db1")
        note_cols.append(f"{uname} notes")
        
        try:
            d[f"{uname} db2"] = DocUserCat.objects.get(doc_id=d["id"], user=u, category__parent_category=362).category.name
        except:
            d[f"{uname} db2"] = "Unrated"
        db2s.append(d[f"{uname} db2"])
        try:
            d[f"{uname} db1"] = DocOwnership.objects.get(doc_id=d["id"], user=u).get_relevant_display()
        except:
            d[f"{uname} db1"] = None
        try:
            d[f"{uname} notes"] = Note.objects.get(doc_id=d["id"], user=u).text
        except:
            d[f"{uname} notes"] = None           
        
    if len(set(db2s))==1:
        db2v = db2s[0]
    else:
        db2v = "Mixed/Maybe"
    d["db2_overall"] = db2v
    doc_dict.append(d)
    
doc_df = pd.DataFrame.from_dict(doc_dict)

doc_df["db1_resolution"] = doc_df.apply(lambda _: '', axis=1)
doc_df["db2_resolution"] = doc_df.apply(lambda _: '', axis=1)

doc_df = doc_df[['id','link','title','content','db1_overall']+db1_cols+['db1_resolution','db2_overall']+db2_cols+['db2_resolution']+note_cols]
doc_df.head(20)

,id,link,title,content,db1_overall,l.berrangford db1,apfisch db1,ee18ivc db1,miriam db1,db1_resolution,db2_overall,l.berrangford db2,apfisch db2,ee18ivc db2,miriam db2,db2_resolution,l.berrangford notes,apfisch notes,ee18ivc notes,miriam notes
0,44683,https://apsis.mcc-berlin.net/scoping/document/...,Raising awareness about climate change in Paci...,Community-based climate change projects in the...,Mixed,Yes,Maybe,Yes,Yes,,Mixed/Maybe,No,No,Yes,No,,None,None,None,None
1,49638,https://apsis.mcc-berlin.net/scoping/document/...,"Translating Climate Change: Adaptation, Resili...",This article examines the translation of key t...,No,No,No,No,No,,Unrated,Unrated,Unrated,Unrated,Unrated,,None,None,None,None
2,64887,https://apsis.mcc-berlin.net/scoping/document/...,A decision framework for Integrated Assessment...,Decision making in the field of air quality an...,No,No,No,No,No,,Unrated,Unrated,Unrated,Unrated,Unrated,,None,None,None,None
3,77756,https://apsis.mcc-berlin.net/scoping/document/...,Quantifying vulnerability of rural communities...,In response to the increasing frequency and ec...,Mixed,Yes,No,No,No,,Mixed/Maybe,No,Unrated,Unrated,Unrated,,None,None,None,None
4,88908,https://apsis.mcc-berlin.net/scoping/document/...,The effect of long-term drying associated with...,Land-use and climate change are expected to ca...,No,No,No,No,No,,Unrated,Unrated,Unrated,Unrated,Unrated,,None,None,None,None
5,96092,https://apsis.mcc-berlin.net/scoping/document/...,From the circular economy to a green economy. ...,The transition to a high level of human develo...,Mixed,No,No,No,Maybe,,Unrated,Unrated,Unrated,Unrated,Unrated,,None,None,None,can’t tell if this is empirical\r\nunder some ...
6,110619,https://apsis.mcc-berlin.net/scoping/document/...,Proteomic Techniques and Management of Floodin...,Climate change is considered a major threat to...,Mixed,No,Maybe,No,Yes,,Mixed/Maybe,Unrated,Maybe,Unrated,No,,None,None,None,None
7,188512,https://apsis.mcc-berlin.net/scoping/document/...,Agro-climatic adaptation of cropping systems u...,Climate change affects the heat and water reso...,Mixed,Yes,No,Maybe,Maybe,,Mixed/Maybe,No,Unrated,Unrated,Unrated,,None,None,None,"assessment, seems like just arguing for an ada..."
8,222427,https://apsis.mcc-berlin.net/scoping/document/...,Heat wave effects on biomass and vegetative gr...,Elevated temperatures and extreme climatic eve...,No,No,No,No,No,,Unrated,Unrated,Unrated,Unrated,Unrated,,None,None,None,non-human adaptation
9,225751,https://apsis.mcc-berlin.net/scoping/document/...,Simplified modelling of apple flowering phenol...,Plant phenology has a strong sensitivity to te...,No,No,No,No,No,,Unrated,Unrated,Unrated,Unrated,Unrated,,None,None,None,None


In [155]:
writer = pd.ExcelWriter("screening-multiuser.xlsx", engine='xlsxwriter')

doc_df.to_excel(writer,index=False, startrow=1, header=False)

workbook  = writer.book
worksheet = writer.sheets['Sheet1']

cbase = ord('A')
tcols = ["id"]
wcols = ["title","content"]

wformat = workbook.add_format({
    'text_wrap': True,
    'valign': "top"
})

yformat = workbook.add_format({
    "fg_color": "#ffffb3"
})

gformat = workbook.add_format({
    "fg_color": "#8dd3c7"
})

bformat = workbook.add_format({
    "fg_color": "#80b1d3"
})

header_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color': 'black',
    'font_color': 'white',
    'border': 1})

for col_num, value in enumerate(doc_df.columns.values):
    worksheet.write(0, col_num , value, header_format)
    
worksheet.set_row(0,25)

for i, c in doc_df.iterrows():
    worksheet.set_row(i,75)

for i, c in enumerate(doc_df.columns):
    l = chr(cbase+i)
    if c == "title":
        worksheet.set_column(f"{l}:{l}",25, wformat)
    elif c == "content":
        worksheet.set_column(f"{l}:{l}",60, wformat)
    elif c in db1_cols:
        worksheet.set_column(f"{l}:{l}", 10, yformat )
    elif c in db2_cols:
        worksheet.set_column(f"{l}:{l}", 10, bformat )
    elif "_overall" in c:
        worksheet.set_column(f"{l}:{l}", 10, gformat )
    elif "notes" in c:
        worksheet.set_column(f"{l}:{l}",20, wformat)

writer.save()

In [106]:
doc_df.to_csv("screening_multiuser.csv",index=False)

In [170]:

docs = Doc.objects.filter(query=q,content__iregex="\w")
doids = DocOwnership.objects.filter(query=q).values_list('doc__id',flat=True)
docs = docs.exclude(pk__in=doids)
docs.count()

tag, created = Tag.objects.get_or_create(
    query=q,
    title="unassigned_with_abstract",
    text="Manually assigned"
)

docs = docs.exclude(tag=tag)

Through = Doc.tag.through
tms = [Through(doc_id=d,tag=tag) for d in set(docs.values_list('pk',flat=True))]
Through.objects.bulk_create(tms)

tag.update_tag()

In [168]:

docs = Doc.objects.filter(query=q,content__iregex="\w")
doids = DocOwnership.objects.filter(query=q,relevant__gt=0).values_list('doc__id',flat=True)
docs = docs.exclude(pk__in=doids)
docs.count()

tag, created = Tag.objects.get_or_create(
    query=q,
    title="unrated_with_abstract",
    text="Manually assigned"
)

docs = docs.exclude(tag=tag)

Through = Doc.tag.through
tms = [Through(doc_id=d,tag=tag) for d in set(docs.values_list('pk',flat=True))]
Through.objects.bulk_create(tms)

tag.update_tag()

56933